In [17]:
%matplotlib notebook
%run models.ipynb
%run preprocess.ipynb

import os
import numpy as np 
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.utils.data as td 
import torchvision as tv
from PIL import Image
import matplotlib.pyplot as plt
import nntools as nt
import PIL
import math
from torch.autograd import Variable

In [36]:
def train_model(model, data, epochs, B=100, gamma=0.01, rho=0.9):
    model.train()
    
    N = len(data) # size of data
    NB = int((N+B-1)/B) # number of minibatches
    criterion = nn.CrossEntropyLoss()
    #optimizer = torch.optim.SGD(model.parameters(), lr=gamma, momentum=rho)
    
    for t in range(epochs):
        running_loss = 0
        shuffled_indices = np.random.permutation(range(N))
        
        for k in range(NB):
            # extract k-th minibatch from xtrain and ltrain
            minibatch_indices = shuffled_indices[B*k:min(B*(k+1), N)]
            
            return minibatch_indices
            

In [31]:
train_loader = 

In [22]:
i, q, a = None, None, None

for a, b, c in train_loader:
    i = a
    q = b
    a = c
    
    break

In [27]:
i = i.cuda()
q = q.cuda()
a = a.cuda()

In [128]:
# %run models.ipynb

# vgg = VGGNet(output_features=1024)
# vgg = vgg.cuda()

# out_vgg = vgg(i)

# lstm = LSTM(len(train.vocab_q), embedding_dim=1000, batch_size=10)
# lstm = lstm.cuda()

# out_lstm = lstm(q)

In [129]:
# pI = torch.rand(10,196,1)
# vI = torch.rand(10,196,1024)
# torch.matmul(vI.view(10, 1024,196),pI).shape

In [130]:
# net = AttentionNet(100,batch_size=10,input_features=1024,output_features=512,image_features=196).to('cuda')
# x = net(out_lstm.to('cuda'), out_vgg.to('cuda'))


In [147]:
%run models.ipynb

san = SAN(num_classes=1000, batch_size=10, vocab_size=len(train.vocab_q), embedding_dim=1000,
          output_vgg=1024, input_attention=1024, output_attention=512)

san = san.cuda()

output = san(i, q)

In [155]:
torch.utils.data.random_split(train, 100000)

TypeError: 'int' object is not iterable

In [179]:
rnge1 = list(range(int(len(train) * 0.8)))
rnge2 = list(range(rnge1[-1]+1, len(train)))

t_ind = torch.utils.data.sampler.SubsetRandomSampler(rnge1)
v_ind = torch.utils.data.sampler.SubsetRandomSampler(rnge2)

In [181]:
train_loader = torch.utils.data.DataLoader(train, batch_size=10, pin_memory=True, sampler=t_ind)
val_loader = torch.utils.data.DataLoader(train, batch_size=10, pin_memory=True, sampler=v_ind)